In [1]:
from rubin_sim.satellite_constellations import Satellite_avoid_basis_function
from rubin_sim.satellite_constellations import Model_observatory
import rubin_sim.satellite_constellations.sat_utils as sat_utils


import healpy as hp
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline


In [ ]:

titles = {'starlink_tles_v1': 'StarlinkV1',
          'starlink_tles_v2': 'StarlinkV2',
          'oneweb_tles': 'OneWeb'}
# Takes a freaking long time to run, because the observatory is predicting things for a full night I think.
for constellation_name in ['starlink_tles_v1', 'starlink_tles_v2', 'oneweb_tles']:

    func = getattr(sat_utils, constellation_name)
    tles = func()
    constellation = sat_utils.Constellation(tles)
    
    mo = Model_observatory(constellation=constellation)
    for tstep in [0, .03]:
        mo.mjd += tstep
        conditions = mo.return_conditions()
        bf = Satellite_avoid_basis_function()
        sat_map = bf(conditions)
        
        below = np.where(conditions.alt < np.radians(20))[0]
        sat_map[below] = np.nan

        title = titles[constellation_name]
        title += ' sun altitude=%.1f degrees' % np.degrees(conditions.sunAlt)
        hp.mollview(sat_map, rot=(conditions.lmst*180/12., conditions.site.latitude),
                    title=title, cbar=False, min=-18, max=0)
        # healpy failing on negative colorbar for some reason. That's a new one.
        ax = plt.gca()
        im = ax.get_images()[0]
        cb = plt.colorbar(im,
                            shrink=0.75,
                            aspect=25,
                            pad=0.1,
                            orientation="horizontal",
                            extendrect=True)
        cb.set_label('unweighted reward (unitless)')
        hp.graticule()
        plt.savefig(constellation_name+'_%.2f_basisfunc.pdf' % tstep)
        

In [ ]:
hp.mollview(sat_map, rot=(conditions.lmst*180/12., conditions.site.latitude),
                    title=title, unit='unweighted reward (unitless)', min=-18, max=0)

In [ ]:
sat_map = bf(conditions)
below = np.where(conditions.alt < np.radians(20))[0]
sat_map[below] = np.nan

In [ ]:
hp.mollview(sat_map, rot=(conditions.lmst*180/12., conditions.site.latitude),
                    title=title, unit='unweighted reward (unitless)', min=-18, max=0)
hp.graticule()

In [ ]:
hp.mollview(sat_map, cbar=False, min=-10)
ax = plt.gca()
im = ax.get_images()[0]
cb = plt.colorbar(im,
                    shrink=0.75,
                    aspect=25,
                    pad=0.1,
                    orientation="horizontal",
                    extendrect=True)
cb.set_label('unweighted reward (unitless)')
#fig.colorbar(ax)
#cbar = fig.colorbar(ax, ticks=[-1, 0, 1], orientation='horizontal')

In [ ]:
help(plt.colorbar)